In [4]:
# Who are the best customers in terms of revenues, profits, transactions/store visits, number of products, etc.?

In [ ]:
# Customers with the highest sales amount
QUERY = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupons', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupons', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
-- Revenue calculation based on cleaned data
    SELECT
        cust_id,
        SUM(sales_amt) AS total_revenue
    FROM
        CleanedData
    GROUP BY
        cust_id
    ORDER BY
        total_revenue DESC
    LIMIT 10;
    """

)

query_job = client.query(QUERY)  # API request
Revenue = query_job.to_dataframe()  # Converts the query results to a DataFrame

# Now you can work with the data as a pandas DataFrame
Revenue

In [ ]:
# Customers with the most transactions amount
QUERY = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupons', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupons', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
-- Transaction calculation based on cleaned data
    SELECT
        cust_id,
        COUNT(*) AS transaction_count
    FROM
        `CleanedData`
    GROUP BY
        cust_id
    ORDER BY
        transaction_count DESC
    LIMIT 10;
    """

)

query_job = client.query(QUERY)  # API request
Transactions = query_job.to_dataframe()  # Converts the query results to a DataFrame

# Now you can work with the data as a pandas DataFrame
Transactions

In [ ]:
# Customers that buy the most amount of different products
QUERY = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupons', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupons', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
-- Best Product calculation based on cleaned data
    SELECT
        cust_id,
        COUNT(DISTINCT prod_id) AS product_count
    FROM
        `CleanedData`
    GROUP BY
        cust_id
    ORDER BY
      product_count DESC
    LIMIT 10;
    """

)

query_job = client.query(QUERY)  # API request
Best_product = query_job.to_dataframe()  # Converts the query results to a DataFrame

# Now you can work with the data as a pandas DataFrame
Best_product

In [ ]:
# Customers with the most amount of store visits
QUERY = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupons', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupons', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
-- Best Visits calculation based on cleaned data
    SELECT
      cust_id,
      COUNT(DISTINCT DATE(trans_dt)) AS visit_count
    FROM
      `CleanedData`
    GROUP BY
      cust_id
    ORDER BY
      visit_count DESC
    LIMIT 10;
    """

)

query_job = client.query(QUERY)  # API request
Best_visits = query_job.to_dataframe()  # Converts the query results to a DataFrame

# Now you can work with the data as a pandas DataFrame
Best_visits